In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e11/sample_submission.csv
/kaggle/input/playground-series-s5e11/train.csv
/kaggle/input/playground-series-s5e11/test.csv


In [2]:
train_csv = pd.read_csv('/kaggle/input/playground-series-s5e11/train.csv')
train_csv.head()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0


In [3]:
train_csv['employment_status'].unique()

array(['Self-employed', 'Employed', 'Unemployed', 'Retired', 'Student'],
      dtype=object)

In [4]:
train_csv['education_level'].unique()

array(['High School', "Master's", "Bachelor's", 'PhD', 'Other'],
      dtype=object)

In [5]:
# Count of each unique col_a per col_b
train_csv.groupby(['education_level', 'loan_paid_back']).size().reset_index(name='count')

,education_level,loan_paid_back,count
0,Bachelor's,0.0,59027
1,Bachelor's,1.0,220579
2,High School,0.0,34938
3,High School,1.0,148654
4,Master's,0.0,18401
5,Master's,1.0,74696
6,Other,0.0,5261
7,Other,1.0,21416
8,PhD,0.0,1873
9,PhD,1.0,9149


In [6]:
# order_map = {'High School': 1, 
#              "Bachelor's": 2, 
#              "Master's": 4, 
#              "Other": 3,
#              "PhD": 5}
# train_csv['education_level_ord'] = train_csv['education_level'].map(order_map)

In [7]:
order_map = {'Unemployed': 1, 
             "Student": 2, 
             "Self-employed": 3, 
             "Employed": 4,
             "Retired": 5}
train_csv['employment_status_ord'] = train_csv['employment_status'].map(order_map)

In [8]:
train_csv

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back,employment_status_ord
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0,3
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0,4
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0,4
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0,4
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593989,593989,23004.26,0.152,703,20958.37,10.92,Female,Single,High School,Employed,Business,C3,1.0,4
593990,593990,35289.43,0.105,559,3257.24,14.62,Male,Single,Bachelor's,Employed,Debt consolidation,F5,1.0,4
593991,593991,47112.64,0.072,675,929.27,14.13,Female,Married,Bachelor's,Employed,Debt consolidation,C1,1.0,4
593992,593992,76748.44,0.067,740,16290.40,9.87,Male,Single,Bachelor's,Employed,Debt consolidation,B2,1.0,4


In [9]:
train_csv = train_csv.drop(['employment_status'], axis=1)
train_csv

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,loan_purpose,grade_subgrade,loan_paid_back,employment_status_ord
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Other,C3,1.0,3
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Debt consolidation,D3,0.0,4
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Debt consolidation,C5,1.0,4
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Debt consolidation,F1,1.0,4
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Other,D1,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
593989,593989,23004.26,0.152,703,20958.37,10.92,Female,Single,High School,Business,C3,1.0,4
593990,593990,35289.43,0.105,559,3257.24,14.62,Male,Single,Bachelor's,Debt consolidation,F5,1.0,4
593991,593991,47112.64,0.072,675,929.27,14.13,Female,Married,Bachelor's,Debt consolidation,C1,1.0,4
593992,593992,76748.44,0.067,740,16290.40,9.87,Male,Single,Bachelor's,Debt consolidation,B2,1.0,4


In [10]:
X_col = [x for x in train_csv.columns if x != 'loan_paid_back' and x != 'id']
y_col = 'loan_paid_back'

X = train_csv[X_col]
y = train_csv[y_col]

In [11]:
from sklearn.model_selection import train_test_split, ParameterSampler

# X : DataFrame or ndarray, y : Series/ndarray
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

In [12]:
unused_col = ['id']

numeric_col = ['annual_income', 'debt_to_income_ratio', 'credit_score', 
               'loan_amount', 'interest_rate']

ohe_categorical_col = ['gender', 'marital_status', 'loan_purpose', 'education_level']

hash_categorical_col = ['grade_subgrade']

output_col = ['loan_paid_back']

In [13]:
X_train

,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,loan_purpose,grade_subgrade,employment_status_ord
239630,44179.89,0.125,673,16809.71,10.88,Male,Single,Bachelor's,Car,C2,3
517613,23987.29,0.100,661,20250.82,11.63,Female,Single,Master's,Debt consolidation,D2,4
20893,28316.41,0.236,691,15797.45,11.92,Male,Married,High School,Debt consolidation,C1,4
242696,45680.63,0.097,676,14251.12,12.98,Female,Married,Bachelor's,Debt consolidation,C2,4
119896,24206.35,0.053,834,25777.19,8.81,Female,Single,Master's,Home,A4,4
...,...,...,...,...,...,...,...,...,...,...,...
234723,30911.29,0.070,650,13254.57,15.13,Female,Divorced,Bachelor's,Business,D2,4
339836,38327.07,0.174,694,25779.89,13.81,Female,Married,High School,Business,C1,4
274056,37381.64,0.104,661,15659.08,12.67,Male,Single,Other,Debt consolidation,D5,4
66391,43948.63,0.081,727,7103.11,12.45,Female,Divorced,Bachelor's,Debt consolidation,C2,3


In [14]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.feature_extraction import FeatureHasher
from scipy import sparse

class SklearnFeatureHasher_stringtoken(BaseEstimator, TransformerMixin):
    """
    Wrap FeatureHasher with input_type='string' but ensure each sample is an iterable
    of strings (here: a single-token list [value]).
    """
    def __init__(self, n_features=32):
        self.n_features = n_features
        self.input_type = 'string'

    def fit(self, X, y=None):
        self._hasher = FeatureHasher(n_features=self.n_features, input_type=self.input_type)
        return self

    def transform(self, X):
        # Accept Series, DataFrame (single column), 1D array, or list
        if isinstance(X, (pd.Series, pd.DataFrame)):
            arr = X.values.ravel()
        else:
            arr = np.asarray(X).ravel()

        # Convert NaNs to string (or whatever sentinel you prefer)
        arr = np.where(pd.isna(arr), '___nan___', arr).astype(str)

        # IMPORTANT: FeatureHasher(input_type='string') expects each sample to be an iterable
        # of strings (e.g. tokens). Wrap each string into a single-token list:
        samples = [[val] for val in arr]

        hashed = self._hasher.transform(samples)   # returns sparse matrix
        if not sparse.isspmatrix_csr(hashed):
            hashed = hashed.tocsr()
        return hashed

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# 3) Create ColumnTransformer - scale numeric cols, passthrough other columns
preprocessor = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False), ohe_categorical_col),
        ('hash_cat', SklearnFeatureHasher_stringtoken(n_features=32), hash_categorical_col),
        ('num', StandardScaler(), numeric_col)
    ]
)

In [18]:
# short example: manual randomized search + early stopping
import numpy as np
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

# --- prepare data (replace with your X, y) ---
# X, y = ...
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full)


# manual fit for early stopping
preprocessor.fit(X_train)
X_train_t = preprocessor.transform(X_train)
X_valid_t = preprocessor.transform(X_valid)
X_test_t  = preprocessor.transform(X_test)

# --- param search space ---
param_dist = {
    "n_estimators": [100, 200, 400, 800],
    "max_depth": [3, 4, 6, 8],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "reg_alpha": [0, 1, 5],
    "reg_lambda": [1, 5, 10]
}

# --- search settings ---
n_iter = 30
sampler = ParameterSampler(param_dist, n_iter=n_iter, random_state=42)

best_score = -np.inf
best_params = None
best_model = None

for i, params in enumerate(sampler, 1):
    model = XGBClassifier(
        objective="binary:logistic",
        use_label_encoder=False,
        eval_metric="auc",
        random_state=42,
        **params
    )
    # use early stopping on validation set
    model.fit(
        X_train_t, y_train,
        eval_set=[(X_valid_t, y_valid)],
        early_stopping_rounds=10,
        verbose=False
    )
    # evaluate using ROC AUC on validation
    preds = model.predict_proba(X_valid_t)[:, 1]
    score = roc_auc_score(y_valid, preds)

    print(f"iter {i:02d}: val_auc={score:.4f} params={params}")
    if score > best_score:
        best_score = score
        best_params = params
        best_model = model

print("\nBest validation AUC:", best_score)
print("Best params:", best_params)

# final evaluation on held-out test set
test_auc = roc_auc_score(y_test, best_model.predict_proba(X_test_t)[:, 1])
print("Test AUC:", test_auc)


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


iter 01: val_auc=0.7932 params={'subsample': 0.6, 'reg_lambda': 10, 'reg_alpha': 1, 'n_estimators': 200, 'max_depth': 4, 'learning_rate': 0.05, 'colsample_bytree': 1.0}


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


iter 02: val_auc=0.7939 params={'subsample': 0.6, 'reg_lambda': 10, 'reg_alpha': 5, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.1, 'colsample_bytree': 1.0}


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


...

iter 28: val_auc=0.8051 params={'subsample': 0.6, 'reg_lambda': 5, 'reg_alpha': 0, 'n_estimators': 400, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 0.8}


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


iter 29: val_auc=0.8047 params={'subsample': 0.6, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 200, 'max_depth': 8, 'learning_rate': 0.1, 'colsample_bytree': 0.6}


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


iter 30: val_auc=0.8058 params={'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha': 1, 'n_estimators': 800, 'max_depth': 3, 'learning_rate': 0.1, 'colsample_bytree': 0.6}

Best validation AUC: 0.8075043269662927
Best params: {'subsample': 1.0, 'reg_lambda': 5, 'reg_alpha': 5, 'n_estimators': 800, 'max_depth': 8, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
Test AUC: 0.8061567459894932


In [19]:
# manual fit for early stopping
preprocessor.fit(X_train)
X_train_t = preprocessor.transform(X_train)
X_val_t   = preprocessor.transform(X_val)
X_test_t  = preprocessor.transform(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [20]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

model = XGBClassifier(
    objective="binary:logistic",
    use_label_encoder=False,    # suppress older warning
    eval_metric="auc",
    random_state=42,
    subsample=1.0, 
    reg_lambda=1, 
    reg_alpha=1, 
    n_estimators=800, 
    min_child_weight=10, 
    max_depth=3, 
    learning_rate=0.1, 
    colsample_bytree=0.6
)

# Fit with validation set and early stopping
model.fit(
    X_train_t, y_train,
    eval_set=[(X_val_t, y_val)],
    early_stopping_rounds=20,
    verbose=True
)

# Predict + evaluate
y_pred = model.predict(X_test_t)
y_proba = model.predict_proba(X_test_t)[:,1]
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))

/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.76263
[1]	validation_0-auc:0.76806
[2]	validation_0-auc:0.77288
[3]	validation_0-auc:0.77984
[4]	validation_0-auc:0.77974
[5]	validation_0-auc:0.78019
[6]	validation_0-auc:0.78002
[7]	validation_0-auc:0.77978
[8]	validation_0-auc:0.77959
[9]	validation_0-auc:0.78030
[10]	validation_0-auc:0.78024
[11]	validation_0-auc:0.78027
[12]	validation_0-auc:0.77977
[13]	validation_0-auc:0.78101
[14]	validation_0-auc:0.78100
[15]	validation_0-auc:0.78129
[16]	validation_0-auc:0.78165
[17]	validation_0-auc:0.78174
[18]	validation_0-auc:0.78155
[19]	validation_0-auc:0.78166
[20]	validation_0-auc:0.78219
[21]	validation_0-auc:0.78219
[22]	validation_0-auc:0.78284
[23]	validation_0-auc:0.78368
[24]	validation_0-auc:0.78370
[25]	validation_0-auc:0.78430
[26]	validation_0-auc:0.78429
[27]	validation_0-auc:0.78435
[28]	validation_0-auc:0.78468
[29]	validation_0-auc:0.78487
[30]	validation_0-auc:0.78513
[31]	validation_0-auc:0.78516
[32]	validation_0-auc:0.78515
[33]	validation_0-au

In [21]:
test_df = pd.read_csv('/kaggle/input/playground-series-s5e11/test.csv')

In [22]:
test_df = test_df.drop(unused_col, axis=1)
test_df

,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade
0,28781.05,0.049,626,11461.42,14.73,Female,Single,High School,Employed,Other,D5
1,46626.39,0.093,732,15492.25,12.85,Female,Married,Master's,Employed,Other,C1
2,54954.89,0.367,611,3796.41,13.29,Male,Single,Bachelor's,Employed,Debt consolidation,D1
3,25644.63,0.110,671,6574.30,9.57,Female,Single,Bachelor's,Employed,Debt consolidation,C3
4,25169.64,0.081,688,17696.89,12.80,Female,Married,PhD,Employed,Business,C1
...,...,...,...,...,...,...,...,...,...,...,...
254564,92835.97,0.068,744,29704.00,13.48,Female,Single,Bachelor's,Employed,Debt consolidation,B2
254565,48846.47,0.091,634,20284.33,9.58,Female,Married,High School,Employed,Debt consolidation,D4
254566,20668.52,0.096,718,26387.55,9.00,Male,Single,Master's,Employed,Debt consolidation,C4
254567,34105.09,0.094,739,11107.36,9.81,Male,Single,Bachelor's,Employed,Business,C2


In [23]:
test_df_t   = preprocessor.transform(test_df)

In [ ]:
test_df_t

In [ ]:
# get probability for positive class
probs = model.predict_proba(test_df_t)[:,1]

In [ ]:
probs

In [ ]:
sub = pd.read_csv('/kaggle/input/playground-series-s5e11/sample_submission.csv')
sub[sub.columns[1]] = probs

In [ ]:
sub

In [ ]:
sub.to_csv('submission_xgboost.csv', index=False)